## airtable_utilites_v1

Ideas (Man there's some many ways to do this!)
- Pull in current house value from ZIllow using Zillow API
- Looks like I can automatically pull in Apple health data: https://www.healthexportapp.com/
- Could have a personal and profressional dashboard if I run out of space
- I would like to make a repo for this.
- Is there an airtable app to show a little bit of a table?
- Add "subgoals complete" to dashboard?
- Force tables up to current day (stop at end of quarter?)

```
jupyter nbconvert --ExecutePreprocessor.timeout=600 --to notebook --execute airtable_utilites_v1.ipynb
```

In [1]:
import logging 
from airtable import *
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    datefmt='%m-%d %H:%M:%S',level='INFO')
logger=logging.getLogger(__name__)
logger.setLevel('INFO')

In [3]:
END_OF_PERIOD=pd.datetime.strptime('2021-6-30', '%Y-%m-%d')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


### Exercise Metrics

In [4]:
logger.info('Connecting to Exercise Log table...')
a=Airtable('appXxfrycEcPpmzgZ', 'Exercise Log', '')

[05-09 20:06:49] p5663 {<ipython-input-4-40334cc72b2c>:1} INFO - Connecting to Exercise Log table...


In [5]:
res=a.get_all()
logger.info('Found %i exercise logs', len(res))
res=[{**r['fields'], **r} for r in res]

[05-09 20:06:50] p5663 {<ipython-input-5-3292140206b1>:2} INFO - Found 50 exercise logs


In [6]:
logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])

df=df.sort_index()
df['Type']=[o[0] for o in df['Exercise Type']]
dfr=df.resample('D').sum() #Resample

[05-09 20:06:50] p5663 {<ipython-input-6-e27dd6a78e81>:1} INFO - Converting to dataframe and resampling...


In [7]:
if datetime.now()>END_OF_PERIOD: end_date=END_OF_PERIOD
else: end_date=pd.datetime.now()
    
# dfr.loc[pd.datetime.strptime('2021-4-20', '%Y-%m-%d')]=0 #Working test case
if end_date not in dfr.index: #Do we have data from today?
    dfr.loc[end_date]=0 #Add 0s at todays date
    dfr=dfr.resample('D').sum()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [8]:
logger.info('Computing cumulative sums by type...')

for s in df['Type'].unique(): dfr['Volume_'+s]=df[df['Type']==s]['Volume'].resample('D').sum()
dfr=dfr.replace(np.NaN, 0)
for s in df['Type'].unique(): dfr['Volume_Cumulative_'+s]=dfr['Volume_'+s].cumsum()

[05-09 20:06:50] p5663 {<ipython-input-8-2266225b9fc1>:1} INFO - Computing cumulative sums by type...


In [9]:
logger.info('Connecting to Exercise Analytics table...')
a2=Airtable('appXxfrycEcPpmzgZ', 'Exercise Analytics', '')

[05-09 20:06:50] p5663 {<ipython-input-9-ebf85e75d19d>:1} INFO - Connecting to Exercise Analytics table...


In [10]:
logger.info('Deleting old Analytics data..')
r=a2.get_all()
ids=[o['id'] for o in r]
res=a2.batch_delete(ids)

[05-09 20:06:50] p5663 {<ipython-input-10-ed77463c2acb>:1} INFO - Deleting old Analytics data..


In [11]:
logger.info('Uploading new analytics data...')
l=[]
for s in df['Type'].unique():
    for d in dfr.index:
        l.append({'Date': str(d.date()),
                 'Exercise Volume': dfr.loc[d]['Volume_Cumulative_'+s],
                 'Exercise Type': s})
res=a2.batch_insert(l)

[05-09 20:06:59] p5663 {<ipython-input-11-c165ed44ade0>:1} INFO - Uploading new analytics data...


## Quality Metrics 

- I think the patters are staring to emerge, it's like an outcome/goal, tactics, and leads. 
- Perhaps I can make a general one of these for any goal??

In [12]:
base_id='appXxfrycEcPpmzgZ'
api_key='' #TODO - put in env
src_table_name='Quality & Habits Log'
dst_table_name='Quality Analytics'
lead_measures=['Mediation', 'Gratitude', 'Affirmations', 'Visualization', 'Reading']

In [13]:
logger.info('Connecting to %s table...', src_table_name)
a=Airtable(base_id, src_table_name, api_key)

res=a.get_all()
logger.info('Found %i records', len(res))
res=[{**r['fields'], **r} for r in res]

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])
df=df.sort_index()
dfr=df.resample('D').sum() #Resample

if datetime.now()>END_OF_PERIOD: 
    logger.info('Period has ended, using %s as end date', END_OF_PERIOD)
    end_date=END_OF_PERIOD
else: 
    end_date=pd.datetime.now()
    logger.info('Using todays date %s as end date', end_date)
    
if end_date not in dfr.index: #Do we have data from today?
    logger.info('No data found from end_date, adding 0 record. ')
    dfr.loc[end_date]=0 #Add 0s at todays date
    dfr=dfr.resample('D').sum()

logger.info('Computing cumulative sums by type...')
for s in lead_measures: dfr['Cumulative_'+s]=dfr[s].cumsum()
dfr=dfr.replace(np.NaN, 0)

logger.info('Connecting to %s table...', dst_table_name)
a=Airtable(base_id, dst_table_name, api_key)

logger.info('Deleting existing %s data..', dst_table_name)
r=a.get_all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

logger.info('Uploading new analytics data...')
l=[]
for s in lead_measures:
    for d in dfr.index:
        l.append({'Date': str(d.date()),
                 'Measure': dfr.loc[d][s],
                 'Cumulative Measure': dfr.loc[d]['Cumulative_'+s],
                 'Tactic': s})
res=a.batch_insert(l)
logger.info('Completed uploading %i records. ', len(res))

[05-09 20:07:08] p5663 {<ipython-input-13-ca3e6e0fe6a9>:1} INFO - Connecting to Quality & Habits Log table...
[05-09 20:07:08] p5663 {<ipython-input-13-ca3e6e0fe6a9>:5} INFO - Found 43 records
[05-09 20:07:08] p5663 {<ipython-input-13-ca3e6e0fe6a9>:8} INFO - Converting to dataframe and resampling...
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
[05-09 20:07:08] p5663 {<ipython-input-13-ca3e6e0fe6a9>:19} INFO - Using todays date 2021-05-09 20:07:08.618818 as end date
[05-09 20:07:08] p5663 {<ipython-input-13-ca3e6e0fe6a9>:22} INFO - No data found from end_date, adding 0 record. 
[05-09 20:07:08] p5663 {<ipython-input-13-ca3e6e0fe6a9>:26} INFO - Computing cumulative sums by type...
[05-09 20:07:08] p5663 {<ipython-input-13-ca3e6e0fe6a9>:30} INFO - Connecting to Quality Analytics table...
[05-09 20:07:08] p5663 {<ipython-input

## Welch Labs Analytics 
- There's some subtle differences between all of these, it will be intersting to see if I can unify...
- TODO: tease out figures drafted vs completed figures

In [14]:
base_id='appjZFu7NntgaSGMj'
api_key='' #TODO - put in env
src_table_names=['References [R]','Correspondence [C]','Figures [F]','Style Examples']
views=['CV Book','CV Book','CV Book','CV Book']
dst_table_name='Research Analytics'
# lead_measures=['References Collected', 'Correspondances']

In [15]:
dfrs=[]
for src_table_name,view in zip(src_table_names, views):
    logger.info('Connecting to %s table...', src_table_name)
    a=Airtable(base_id, src_table_name, api_key)

    res=a.get_all(view=view)
    logger.info('Found %i records', len(res))
    if len(res)==0: continue
    res=[{'id':r['id'], 'createdTime':r['createdTime'], src_table_name:1} for r in res]

    logger.info('Converting to dataframe and resampling...')
    df=pd.DataFrame(res)
    df.index=pd.DatetimeIndex(df['createdTime'])
    dfr=df.resample('D').sum() #Resample
    dfrs.append(dfr)

dfrs=pd.concat(dfrs, sort=False)
dfrs=dfrs.resample('D').sum() #Handle 2:1 case
dfrs=dfrs.sort_index()

if datetime.now()>END_OF_PERIOD: 
    logger.info('Period has ended, using %s as end date', END_OF_PERIOD)
    end_date=END_OF_PERIOD
else: 
    end_date=pd.datetime.now(pytz.utc)
    logger.info('Using todays date %s as end date', end_date)
    
if end_date not in dfr.index: #Do we have data from today?
    logger.info('No data found from end_date, adding 0 record. ')
    dfrs.loc[end_date]=0 #Add 0s at todays date
    dfrs=dfrs.resample('D').sum()

logger.info('Computing cumulative sum...')
dfrs=dfrs.replace(np.NaN, 0) 
for src_table_name in src_table_names: dfrs['Cumulative ' + src_table_name]=dfrs[src_table_name].cumsum()
dfrs=dfrs.replace(np.NaN, 0) 

    
logger.info('Connecting to %s table...', dst_table_name)
a=Airtable(base_id, dst_table_name, api_key)

logger.info('Deleting existing %s data..', dst_table_name)
r=a.get_all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

logger.info('Uploading new analytics data...')
l=[]
for src_table_name in src_table_names:
    for d in dfrs.index:
        l.append({'Date': str(d.date()),
                 'Measure': dfrs.loc[d][src_table_name],
                 'Cumulative Measure': dfrs.loc[d]['Cumulative ' + src_table_name],
                 'Tactic': src_table_name})
res=a.batch_insert(l)
logger.info('Completed uploading %i records. ', len(res))

[05-09 20:07:24] p5663 {<ipython-input-15-ab11ab11b01b>:3} INFO - Connecting to References [R] table...
[05-09 20:07:24] p5663 {<ipython-input-15-ab11ab11b01b>:7} INFO - Found 46 records
[05-09 20:07:24] p5663 {<ipython-input-15-ab11ab11b01b>:11} INFO - Converting to dataframe and resampling...
[05-09 20:07:24] p5663 {<ipython-input-15-ab11ab11b01b>:3} INFO - Connecting to Correspondence [C] table...
[05-09 20:07:25] p5663 {<ipython-input-15-ab11ab11b01b>:7} INFO - Found 7 records
[05-09 20:07:25] p5663 {<ipython-input-15-ab11ab11b01b>:11} INFO - Converting to dataframe and resampling...
[05-09 20:07:25] p5663 {<ipython-input-15-ab11ab11b01b>:3} INFO - Connecting to Figures [F] table...
[05-09 20:07:25] p5663 {<ipython-input-15-ab11ab11b01b>:7} INFO - Found 8 records
[05-09 20:07:25] p5663 {<ipython-input-15-ab11ab11b01b>:11} INFO - Converting to dataframe and resampling...
[05-09 20:07:25] p5663 {<ipython-input-15-ab11ab11b01b>:3} INFO - Connecting to Style Examples table...
[05-09 20

## Welch Labs Writing


In [16]:
base_id='appjZFu7NntgaSGMj'
api_key='' #TODO - put in env
src_table_name='Writing Log'
dst_table_name='Writing Analytics'
lead_measures=['Words Drafted']

In [17]:
logger.info('Connecting to %s table...', src_table_name)
a=Airtable(base_id, src_table_name, api_key)

res=a.get_all()
logger.info('Found %i records', len(res))
res=[{**r['fields'], **r} for r in res]

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])
df=df.sort_index()
dfr=df.resample('D').sum() #Resample

if datetime.now()>END_OF_PERIOD: 
    logger.info('Period has ended, using %s as end date', END_OF_PERIOD)
    end_date=END_OF_PERIOD
else: 
    end_date=pd.datetime.now()
    logger.info('Using todays date %s as end date', end_date)
    
if end_date not in dfr.index: #Do we have data from today?
    logger.info('No data found from end_date, adding 0 record. ')
    dfr.loc[end_date]=0 #Add 0s at todays date
    dfr=dfr.resample('D').sum()
    
dfr=dfr.replace(0, np.nan).ffill() #Metric is already cumulative!

logger.info('Computing cumulative sums by type...')
for s in lead_measures: dfr['Cumulative_'+s]=dfr[s] #.cumsum()
dfr=dfr.replace(np.NaN, 0)

logger.info('Connecting to %s table...', dst_table_name)
a=Airtable(base_id, dst_table_name, api_key)

logger.info('Deleting existing %s data..', dst_table_name)
r=a.get_all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

logger.info('Uploading new analytics data...')
l=[]
for s in lead_measures:
    for d in dfr.index:
        l.append({'Date': str(d.date()),
                 'Measure': float(dfr.loc[d][s]),
                 'Cumulative Measure': float(dfr.loc[d]['Cumulative_'+s]),
                 'Tactic': s})
res=a.batch_insert(l)
logger.info('Completed uploading %i records. ', len(res))

[05-09 20:07:36] p5663 {<ipython-input-17-927b06bb3d86>:1} INFO - Connecting to Writing Log table...
[05-09 20:07:36] p5663 {<ipython-input-17-927b06bb3d86>:5} INFO - Found 18 records
[05-09 20:07:36] p5663 {<ipython-input-17-927b06bb3d86>:8} INFO - Converting to dataframe and resampling...
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
[05-09 20:07:36] p5663 {<ipython-input-17-927b06bb3d86>:19} INFO - Using todays date 2021-05-09 20:07:36.813470 as end date
[05-09 20:07:36] p5663 {<ipython-input-17-927b06bb3d86>:22} INFO - No data found from end_date, adding 0 record. 
[05-09 20:07:36] p5663 {<ipython-input-17-927b06bb3d86>:28} INFO - Computing cumulative sums by type...
[05-09 20:07:36] p5663 {<ipython-input-17-927b06bb3d86>:32} INFO - Connecting to Writing Analytics table...
[05-09 20:07:36] p5663 {<ipython-input-17-927b0

## Satisfaction & Energy

In [18]:
base_id='appXxfrycEcPpmzgZ'
api_key='' #TODO - put in env
src_table_name='Quality & Habits Log'
dst_table_name='Quality Analytics 2'
measures=['Satisfaction', 'Energy']

In [19]:
logger.info('Connecting to %s table...', src_table_name)
a=Airtable(base_id, src_table_name, api_key)

res=a.get_all()
logger.info('Found %i records', len(res))
res=[{**r['fields'], **r} for r in res]

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])
df=df.sort_index()
dfr=df.resample('D').mean() #Resample

logger.info('Connecting to %s table...', dst_table_name)
a=Airtable(base_id, dst_table_name, api_key)

logger.info('Deleting existing %s data..', dst_table_name)
r=a.get_all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

logger.info('Uploading new analytics data...')
l=[]
for s in measures:
    for d in dfr.index:
        if np.isnan(dfr.loc[d][s]): continue
        l.append({'Date': str(d.date()),
                 'Measure': dfr.loc[d][s],
                 'Metric': s})
res=a.batch_insert(l)
logger.info('Completed uploading %i records. ', len(res))

[05-09 20:07:39] p5663 {<ipython-input-19-ee9662896fda>:1} INFO - Connecting to Quality & Habits Log table...
[05-09 20:07:40] p5663 {<ipython-input-19-ee9662896fda>:5} INFO - Found 43 records
[05-09 20:07:40] p5663 {<ipython-input-19-ee9662896fda>:8} INFO - Converting to dataframe and resampling...
[05-09 20:07:40] p5663 {<ipython-input-19-ee9662896fda>:14} INFO - Connecting to Quality Analytics 2 table...
[05-09 20:07:40] p5663 {<ipython-input-19-ee9662896fda>:17} INFO - Deleting existing Quality Analytics 2 data..
[05-09 20:07:43] p5663 {<ipython-input-19-ee9662896fda>:22} INFO - Uploading new analytics data...
[05-09 20:07:45] p5663 {<ipython-input-19-ee9662896fda>:31} INFO - Completed uploading 70 records. 


### Upload Writing File

In [4]:
base_id='appjZFu7NntgaSGMj'
api_key='' #TODO - put in env
src_table_name='Writing Log'
writing_file='/Users/stephenwelch/Dropbox (Personal)/writing/Computer Vision Book.md'

In [5]:
logger.info('Connecting to %s table...', src_table_name)
a=Airtable(base_id, src_table_name, api_key)

res=a.get_all()
logger.info('Found %i records', len(res))
res=[{**r['fields'], **r} for r in res]

dt=datetime.strftime(datetime.now(), '%Y-%m-%d')

for r in res: 
    if r['Date']==dt:
        logger.info('Found Record %s from todays date %s, deleting...', r['id'], r['Date'])
        a.delete(r['id'])
        
logger.info('Opening %s', writing_file)
with open(writing_file, 'r') as f:
    t=f.read()
    
main_text=t.split('The End.')[0]
words_drafted=len(main_text.replace('\n', '').split(' '))
logger.info('Computed %i words drafted', words_drafted)

logger.info('Uploading new record...')
new_record={'Date':dt, 'Words Drafted': words_drafted, 'Project': 'CV Book', 'Full Text': t}
res=a.insert(new_record)

[05-09 21:12:53] p6059 {<ipython-input-5-16d1bdeeb8c8>:1} INFO - Connecting to Writing Log table...
[05-09 21:12:53] p6059 {<ipython-input-5-16d1bdeeb8c8>:5} INFO - Found 19 records
[05-09 21:12:53] p6059 {<ipython-input-5-16d1bdeeb8c8>:12} INFO - Found Record recgclbuwx956IUFQ from todays date 2021-05-09, deleting...
[05-09 21:12:54] p6059 {<ipython-input-5-16d1bdeeb8c8>:15} INFO - Opening /Users/stephenwelch/Dropbox (Personal)/writing/Computer Vision Book.md
[05-09 21:12:54] p6059 {<ipython-input-5-16d1bdeeb8c8>:21} INFO - Computed 2234 words drafted
[05-09 21:12:54] p6059 {<ipython-input-5-16d1bdeeb8c8>:23} INFO - Uploading new record...
